# Definir archivos de lectura de datos y output 

In [1]:
data_train='input/databinarystudents.csv'
data_test='datasetx.csv'
nombre_output='output_binaria.csv'

# Librerias necesarias

In [2]:
import pandas as pd
import numpy as np
from sklearn.preprocessing import StandardScaler
from sklearn.pipeline import Pipeline
from sklearn.compose import ColumnTransformer
from sklearn.impute import SimpleImputer
from sklearn.linear_model import LogisticRegression

# Lectura datos entrenamiento

In [3]:
data=pd.read_csv(data_train)

# Entrenamiento Modelo

In [4]:
sels=['x4', 'x10', 'x17', 'x18', 'x23', 'x24']
X_train=data.filter(sels, axis=1)
Y_train=data['yL']
num_features = ['x10','x23','x24']
num_pipeline = Pipeline([
    ('scaler',StandardScaler()),
    ('imputer',SimpleImputer(strategy='median'))
    ])
col_transformer = ColumnTransformer(transformers=[
    ('num', num_pipeline, num_features)
    ])
col_transformer.fit(X_train.fillna(0))
X_train[num_features]=col_transformer.transform(X_train.fillna(0))
final_model = LogisticRegression(C=1, penalty='elasticnet', solver='saga',
                                 l1_ratio=0.5, max_iter=10000, random_state=0)
final_model.fit(X_train, Y_train)

sels.append('int')

pd.DataFrame({'var':np.array(sels),'coef':np.append(final_model.coef_,final_model.intercept_)})


,var,coef
0,x4,-0.571804
1,x10,0.823720
2,x17,-1.889061
3,x18,-1.293392
4,x23,0.514480
5,x24,-0.419626
6,int,1.738903


# Testeo

In [5]:
test=pd.read_csv(data_test)
sels=['x4', 'x10', 'x17', 'x18', 'x23', 'x24']
test=test.filter(sels, axis=1)
test[num_features]=col_transformer.transform(test.fillna(0))
y_pred_test = final_model.predict_proba(test)[:,1]
y_pred_test1 = final_model.predict(test)
pd.DataFrame({'y_pred':y_pred_test1,'y_prob':y_pred_test}).to_csv(nombre_output)